In [ ]:
# 이력서 파싱 LLM
# 이력서를 LLM에게 제공해서 JSON을 요청하고
# JSON 형식인지 검사하는 LLM에게 요청해서 보정한다 (될 때까지 수차례 반복)
from dotenv import load_dotenv

# LLM 디버깅 설정
load_dotenv('./../.env')

True

In [3]:
from langchain_community.document_loaders import PyMuPDFLoader

In [14]:
# filename = 'resume-2.pdf'
filename = 'resume-1.pdf'

loader = PyMuPDFLoader('resume/{}'.format(filename))
docs = loader.load()

In [15]:
print(docs[0].page_content)

Kumar Pallav
Email : me@kumarpallav.com
http://kumarpallav.com
Mobile : +1-206-910-0006
EDUCATION
Indian Institute of Technology, Bombay
Mumbai, India
Bachelor of Computer Science and Engineering (with Hons.); CGPA 8.6/10
Jun 2010 - May 2014
EXPERIENCE
Microsoft
Redmond, WA
Software Engineer · OneNote
Jun 2016 - Present
◦Magic Ink and Ink Lookup: Worked on recognizing ink strokes into words (using onboard IA) and recognizing
keywords/entities given the context (using custom entity recognition service). Implemented accumulation of images or
interactive widgets (from various online sources) related to the keyword, which could be inserted onto the Whiteboard
canvas.
◦Whiteboard App: Implemented sharing experience for the new collaboration focused Whiteboard app. Worked on
creating and joining a shared session via OneDrive for Business storage. Also added new XAML components to handle
dialogs within the app.
◦Content Addins: Implemented content addins, for OneNote desktop app, as small emb

In [16]:
# pdf의 전체 내용을 context 변수에 담는다
context = docs[0].page_content

# 질문 프롬프트를 작성한다
question = """You are tasked with parsing a job resume. Your goal is to extract relevant information in a valid structured 'JSON' format.
Do not write preambles or explanations."""

In [17]:
# scripts 폴더 안에 llm 파일에서 ask_llm 함수와 validate_json 함수를 가져온다
from scripts.llm import ask_llm, validate_json

In [37]:
response = ask_llm(context=context, question=question)

In [38]:
# JSON 형식으로 요청했지만 완전한 JSON으로 주지 않았다
print(response)

**1. Contact Information:**

- Name: Kumar Pallav
- Email: me@kumarpallav.com
- Phone Number: +1-206-910-0006
- Website/Portfolio: http://kumarpallav.com

**2. Education:**

- Institution Name: Indian Institute of Technology, Bombay
- Degree: Bachelor of Computer Science and Engineering (with Hons.)
- Field of Study: None specified
- Graduation Dates: Jun 2010 - May 2014

**3. Experience:**

- Job Title: Software Engineer
- Company Name: Microsoft
- Location: Redmond, WA, Vancouver, Canada
- Dates of Employment:
    - Software Engineer · OneNote (Jun 2016 - Present)
    - Software Engineer · OneNote (Nov 2014 - May 2016)
- Responsibilities/Projects:

**4. Projects:**

- Project Title: Course Allotment Simulation using SAT Solver
- Description/Technologies Used: SAT (Satisﬁability) problem, heuristics on MiniSAT and Glucose solvers
- Outcomes/Results:
    - Found a best course allocation under various constraints as a SAT problem
    - Studied the relative efﬁciency of various heuristic

In [39]:
# JSON 검증용 LLM 함수를 동작시킨다
response_json = validate_json(response)

In [40]:
print(response_json)

{'Contact Information': {'Name': 'Kumar Pallav', 'Email': 'me@kumarpallav.com', 'Phone Number': '+1-206-910-0006', 'Website/Portfolio': 'http://kumarpallav.com'}, 'Education': {' Institution Name': 'Indian Institute of Technology, Bombay', 'Degree': 'Bachelor of Computer Science and Engineering (with Hons.)', 'Field of Study': None, 'Graduation Dates': ['Jun 2010', 'May 2014']}, 'Experience': {'Job Title': 'Software Engineer', 'Company Name': 'Microsoft', 'Location': 'Redmond, WA, Vancouver, Canada', 'Dates of Employment': [{'Start Date': 'Jun 2016', 'End Date': 'Present'}, {'Start Date': 'Nov 2014', 'End Date': 'May 2016'}]}, 'Projects': {'Project Title': 'Course Allotment Simulation using SAT Solver', 'Description/Technologies Used': 'SAT (Satisﬁability) problem, heuristics on MiniSAT and Glucose solvers', 'Outcomes/Results': [{'Best Course Allocation': 'Under various constraints as a SAT problem'}, {'Relative Eﬃciency of Heuristics': 'Studied the relative efﬃciency of various heuris

In [42]:
# json 파일로 저장해서 확인해본다
import json

output_file = filename.replace('.pdf', '.json')
output_file = 'parsed_resume/{}'.format(output_file)

json.dump(response_json, open(output_file, 'w'), indent=4)